# Week 10. 웹 크롤링 실전
-------------------

영화에 대한 트윗을 보고 이게 긍정적인 평가인지 부정적인 평가인지 판단해보자

- 크롤링 대상 : 네이버 영화 네테즌 평점, 네티즌 리뷰

# Import

In [1]:
import numpy as np
import pandas as pd
import requests
import urllib
from bs4 import BeautifulSoup

# 시작은 미약하게
나중에는 `scrapy` 나 `seleneum` 을 이용해서 `page` 인자를 바꿔가면서 계속 크롤링을 해야 함.

In [3]:
params = urllib.parse.urlencode({"page": 1})
print(params)
url = 'http://movie.naver.com/movie/point/af/list.nhn?&%s' % params
print(url)

page=1
http://movie.naver.com/movie/point/af/list.nhn?&page=1


In [4]:
params = {'page': 1}
url = 'http://movie.naver.com/movie/point/af/list.nhn'
res = requests.get(url, params=params)

In [15]:
print(res.content.decode('euc-kr')[5000:5500])

an></a>
            <a href="#content" id="gnb_goContent" onclick="document.getElementById('content').tabIndex=-1;document.getElementById('content').focus();return false;"><span>본문으로 바로가기</span></a>
    </div>
    <!-- //skip navigation -->
	<!-- GNB -->
	<div class="gnb_container">
		<div class="gnb_content">
			<div class="gnb_box">
			 	<div class="gnb_wrap">
					<div id="gnb" onload="javascript:getGNB();">
				   	 	<script type="text/javascript" charset="euc-kr" src="http://stati


## BeautifulSoup으로 평점 테이블을 파싱

In [16]:
navigator = BeautifulSoup(res.text, 'html.parser')

In [23]:
table = navigator.find('table', class_='list_netizen')

# print(table)
df_table = pd.read_html(str(table), header=0)
df_table[0]

,번호,평점,140자평,글쓴이·날짜,Unnamed: 4
0,12841535,NaN,8,인보카머스 밋밋한 마무리가 아쉽다.. 신고,ykju****17.07.16
1,12841534,NaN,5,하루 오버스러운연출. 영화인걸 감안해도 진정 답답한스토리. 보면서 속으로 욕했다. 신고,jung****17.07.16
2,12841533,NaN,9,옥자 많은 걸 생각하게 함 재밌음 신고,yumm****17.07.16
3,12841532,NaN,10,스파이더맨: 홈커밍 꿀잼이에요 시간가는줄모른듯 신고,love****17.07.16
4,12841531,NaN,1,분노의 질주: 더 익스트림 정말 다 좋은데. 엘레나 너무 불쌍하다!!! 보고나서 찝...,1106****17.07.16
5,12841530,NaN,7,설계 오인혜는 몸뚱이 없었으면 어쩔뻔했누..쓸데없는 배드신마저 연기 드럽게 못하네....,nari****17.07.16
6,12841529,NaN,9,혹성탈출 마지막 진짜 반전이였다 . 신고,rlag****17.07.16
7,12841528,NaN,10,내 친구의 집은 어디인가 마음을 순수하게 해주는 영화라서 별 10개 준다. 이 영화...,whit****17.07.16
8,12841527,NaN,8,"러브, 로지 둘이 너무 잘어울린다 ㅜㅜ 근데 왜 계속 방해하는거야 스토리는 보금 아...",rlag****17.07.16
9,12841526,NaN,7,"옥자 중간에 약간 지루한 타임이 있지만, 볼만 합니다. 신고",mari****17.07.16


> 평점에서 하나씩 밀리네...  조금더 정형화 되어 있는 데이터로 만들려면 customization 이 필요

In [24]:
import dateutil

list_records = []

for i, r in enumerate(table.find_all('tr')):
    for j, c in enumerate(r.find_all('td')):
        if j == 0:
            record = {"번호": int(c.text.strip())}
        elif j == 2:
            record.update({"평점": int(c.text.strip())})
        elif j == 3:
            record.update({"영화": str(c.find('a', class_='movie').text.strip())})
            record.update({"140자평": str(c.find('br').text.strip()).split('\r')[0]})
        elif j == 4:
            record.update({"글쓴이": c.find('a', class_='author').text.strip()})
            record.update({"날짜": dateutil.parser.parse(c.find('br').text.strip(), yearfirst=True)})
    try:        
        list_records.append(record)
    except:
        pass
    
list_records[:2]

[{'140자평': '밋밋한 마무리가 아쉽다.. ',
  '글쓴이': 'ykju****',
  '날짜': datetime.datetime(2017, 7, 16, 0, 0),
  '번호': 12841535,
  '영화': '인보카머스',
  '평점': 8},
 {'140자평': '오버스러운연출. 영화인걸 감안해도 진정 답답한스토리.  보면서 속으로 욕했다. ',
  '글쓴이': 'jung****',
  '날짜': datetime.datetime(2017, 7, 16, 0, 0),
  '번호': 12841534,
  '영화': '하루',
  '평점': 5}]

## Dataframe으로 

In [6]:
df = pd.DataFrame(list_records, 
                  columns=["번호", "평점", "영화", "140자평", "글쓴이", "날짜"])
df.head(30)

,번호,평점,영화,140자평,글쓴이,날짜
0,12817499,9,스파이더맨: 홈커밍,너무너무 재미있어요 오랜만에 제대로된 영화 본거같아요,horr****,2017-07-08
1,12817498,1,스파이더맨: 홈커밍,초반에 휴대폰 이리저리 바꿔가면서 영상촬영하는 장면에서 어지러워서 나왔어요 영화가 ...,zldn****,2017-07-08
2,12817497,7,스파이더맨: 홈커밍,평점 왜케높지..어린이영화보고온듯,fari****,2017-07-08
3,12817496,5,악녀,기대했지만 역시나 기대만..여배우 고생 한것 같고..,wcwn****,2017-07-08
4,12817495,10,스파이더맨: 홈커밍,단순한 액션물을 보러 오신거라면 그대로 다시 집으로돌아가세요,chee****,2017-07-08
5,12817494,1,여교수의 은밀한 매력,짱구야...너 그런 애였니?,wow1****,2017-07-08
6,12817493,3,아티스트 봉만대,"멜로 영화인데, 왜 다큐로 느꺼지지.",mahu****,2017-07-08
7,12817492,10,헤드윅,"슬픈영환데, 왜 이렇게 기분 좋죠....특이하네",vivi****,2017-07-08
8,12817491,9,원더 우먼,원더우먼에 대해 새로운 느낌...,whit****,2017-07-08
9,12817490,10,겟 아웃,대박 영화 ㅠㅠ 재밋었습니다,mytn****,2017-07-08
